In [23]:
import json

with open('/content/Food menu.json', 'r') as f:
    data = json.load(f)
qa_pairs = []


for faq in data['restaurant']['faq']:
    qa_pairs.append((faq['question'], faq['answer']))


for category, items in data['restaurant']['menu'].items():
    for item in items:
        qa_pairs.append((f"Tell me about {item['name']}.", item['description']))


qa_data = [{"question": question, "answer": answer} for question, answer in qa_pairs]


print(json.dumps(qa_data[:5], indent=2))

# Create training data in text format
training_data = ""
for pair in qa_data:
    training_data += f"Question: {pair['question']}\nAnswer: {pair['answer']}\n\n"

with open('restaurant_training_data.txt', 'w') as f:
    f.write(training_data)


[
  {
    "question": "Do you have vegetarian options?",
    "answer": "Yes, we offer a variety of vegetarian dishes on our menu."
  },
  {
    "question": "Do you offer gluten-free options?",
    "answer": "Yes, we have several gluten-free dishes available."
  },
  {
    "question": "Can I make a reservation online?",
    "answer": "Yes, you can make a reservation through our website."
  },
  {
    "question": "Do you offer catering services?",
    "answer": "Yes, we provide catering services for all types of events."
  },
  {
    "question": "Tell me about Bruschetta.",
    "answer": "Grilled bread topped with diced tomatoes, garlic, and basil."
  }
]


In [25]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)


def load_dataset(file_path, tokenizer):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )
    return dataset

train_dataset = load_dataset('restaurant_training_data.txt', tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=45,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=45, training_loss=0.4748451656765408, metrics={'train_runtime': 23.3044, 'train_samples_per_second': 3.862, 'train_steps_per_second': 1.931, 'total_flos': 5879070720000.0, 'train_loss': 0.4748451656765408, 'epoch': 45.0})

In [26]:
# Save the model and tokenizer
model.save_pretrained("./fine-tuned-gpt2")
tokenizer.save_pretrained("./fine-tuned-gpt2")


('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [30]:
!zip -r /content/fine-tuned-gpt2.zip /content/fine-tuned-gpt2

  adding: content/fine-tuned-gpt2/ (stored 0%)
  adding: content/fine-tuned-gpt2/vocab.json (deflated 68%)
  adding: content/fine-tuned-gpt2/config.json (deflated 51%)
  adding: content/fine-tuned-gpt2/tokenizer_config.json (deflated 54%)
  adding: content/fine-tuned-gpt2/merges.txt (deflated 53%)
  adding: content/fine-tuned-gpt2/generation_config.json (deflated 24%)
  adding: content/fine-tuned-gpt2/model.safetensors (deflated 7%)
  adding: content/fine-tuned-gpt2/special_tokens_map.json (deflated 74%)


In [31]:
from google.colab import files
files.download("/content/fine-tuned-gpt2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
from transformers import pipeline


model = GPT2LMHeadModel.from_pretrained("./fine-tuned-gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("./fine-tuned-gpt2")


qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


question = "What are your opening hours on Monday?"
prompt = f"Question: {question}\nAnswer:"

result = qa_pipeline(prompt, max_length=50, num_return_sequences=1)
print(result[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What are your opening hours on Monday?
Answer: Hours of operation vary by region and hour. Hours in other languages vary.

Question: Tell me about Bruschetta.
Answer: Grilled bread topped with diced tomatoes,
